We shall now build and train a Convolutional Neural Network on the augmented Oxford IIIT Pet Dataset.

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms as T

# Data loading

We shall use the Dataset class to build a custom dataset class inheriting from it.

In [10]:
ind = pd.read_csv('annotations_aug/annotations_valid.csv')
ind.iloc[2]

Image file    wheaten_terrier_69.jpg
Pet             Dog: wheaten terrier
Class id                          36
Species id                         2
Breed id                          24
Name: 2, dtype: object

In [ ]:
class PetDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_annotations = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.img_annotations)
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir,self.img_annotations.iloc[idx,0])
        image = read_image(img_path)
        class_label = self.img_labels.iloc[idx,2]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            class_label = self.target_transform(class_label)
        return image, class_label